In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import torch, torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Normalize, Resize, RandomHorizontalFlip

In [20]:
transform_train = transforms.Compose([
    Resize(size=(224,224)),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    Resize(size=(224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
from mydataset import MyDataset
cifa10_dataset = MyDataset(
    csv_file_dir="./cifar-10/trainLabels.csv",
    images_dir="./cifar-10/train",
    transform=transform_train
)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [22]:
from torch.utils.data import random_split, DataLoader


In [23]:
N = len(cifa10_dataset)

In [24]:
train_size = int(0.8 * N)
val_size = N - train_size

train_dataset, val_dataset = random_split(
    dataset=cifa10_dataset,
    lengths=[train_size, val_size]
)
len(train_dataset)

40000

In [25]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)


In [26]:

import torchvision.models as models

In [27]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
model = models.vgg16()
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [29]:
# for param in model.features.parameters():
#     param.requires_grad = False
# for param in model.features[24:].parameters():
#     param.requires_grad = True

In [30]:
import torch.nn as nn
model.classifier[6] = nn.Linear(
    in_features=model.classifier[6].in_features,
    out_features=10
)

In [31]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [33]:
from tqdm.notebook import tqdm
import torch
EPOCHS = 16
epochs_tqdm = tqdm(range(EPOCHS), unit='epoch', desc='Training')
model = model.to('mps')
for epoch in epochs_tqdm:
    num_train_correct = 0
    num_item = 0
    model.train()
    train_tqdm = tqdm(enumerate(train_loader), unit='batch', total=len(train_loader))
    for i, (images, labels) in train_tqdm:
        images = images.to('mps')
        labels = labels.to('mps')
        optimizer.zero_grad()
        out = model(images)
        loss = criterion(out, labels)


        loss.backward()
        optimizer.step()

        _,predicts = out.max(dim=1)
        num_train_correct += (predicts == labels).sum().item()
        num_item += len(predicts)
        if i % 100 == 0:
            train_tqdm.set_postfix(train_acc = num_train_correct/num_item)

    num_test_correct = 0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to('mps')
            labels = labels.to('mps')
            out = model(images)
            _,predicts = out.max(dim=1)
            num_test_correct += (predicts == labels).sum().item()
    epochs_tqdm.set_postfix(val_acc = num_test_correct/len(val_loader))




Training:   0%|          | 0/16 [00:00<?, ?epoch/s]

  0%|          | 0/625 [00:00<?, ?batch/s]

KeyboardInterrupt: 